# Recuperatorio Segundo Parcial Práctica

In [1]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\joi\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [3]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

Kernel added: #!sql-Pampero

1)	Listar los clientes del Mercosur (Argentina, Brasil, Paraguay y Uruguay) mostrando ID, nombre de la empresa, dirección completa (concatenando Dirección, Ciudad, Región y CódigoPostal), y país. Ordenado por país y código postal.

In [4]:
SELECT IDCliente, NombreEmpresa, CONCAT_WS(',', Direccion, ciudad, region, CodigoPostal) as DireccionCompleta, Pais
  FROM Clientes
  WHERE pais IN ('Argentina', 'Brasil', 'Paraguay', 'Uruguay')
  ORDER BY Pais, CodigoPostal;


(12 filas afectadas)

IDCliente,NombreEmpresa,DireccionCompleta,Pais
CACTU,Cactus Comidas para llevar,"Cerrito 333,Buenos Aires,1010",Argentina
RANCH,Rancho grande,"Av. del Libertador 900,Buenos Aires,1010",Argentina
OCEAN,Océano Atlántico Ltda.,"Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,1010",Argentina
QUEDE,Que Delícia,"Rua da Panificadora, 12,Rio de Janeiro,RJ,02389-673",Brasil
RICAR,Ricardo Adocicados,"Av. Copacabana, 267,Rio de Janeiro,RJ,02389-890",Brasil
GOURL,Gourmet Lanchonetes,"Av. Brasil, 442,Campinas,SP,04876-786",Brasil
COMMI,Comércio Mineiro,"Av. dos Lusíadas, 23,Sao Paulo,SP,05432-043",Brasil
FAMIA,Familia Arquibaldo,"Rua Orós, 92,Sao Paulo,SP,05442-030",Brasil
HANAR,Hanari Carnes,"Rua do Paço, 67,Rio de Janeiro,RJ,05454-876",Brasil
QUEEN,Queen Cozinha,"Alameda dos Canàrios, 891,Sao Paulo,SP,05487-020",Brasil


2)	Identifique los 3 productos con la mayor cantidad total vendida (Cantidad) dentro las categorías Mariscos y Carnes. Muestre el nombre de la categoría, el nombre del producto y la cantidad total vendida, asegurándose de manejar correctamente los empates (si dos productos empatan, ambos deberían ser incluidos si caen dentro del top 3).

In [5]:
WITH VentasPorProducto AS (
    -- 1. Agregación de cantidad total vendida por producto y categoría
    SELECT
        P.NombreProducto,
        C.NombreCategoria,
        SUM(DP.Cantidad) AS CantidadTotalVendida,
        -- 2. Aplicación de ranking dentro de cada categoría
        RANK() OVER (
            PARTITION BY C.IDCategoria
            ORDER BY SUM(DP.Cantidad) DESC
        ) AS RangoVenta
    FROM
        Productos AS P
    INNER JOIN Categorias AS C ON P.IDCategoria = C.IDCategoria
    INNER JOIN [Detalles Pedido] AS DP ON P.IDProducto = DP.IDProducto
    GROUP BY
        P.IDProducto, P.NombreProducto, C.IDCategoria, C.NombreCategoria
)
-- 3. Filtrado para obtener solo los 3 principales
SELECT
    NombreCategoria,
    NombreProducto,
    CantidadTotalVendida
FROM
    VentasPorProducto
WHERE
    RangoVenta <= 3
    AND NombreCategoria IN ('Mariscos', 'Carnes')
ORDER BY
    NombreCategoria, RangoVenta;


(6 filas afectadas)

NombreCategoria,NombreProducto,CantidadTotalVendida
Carnes,Alice Mutton,978
Carnes,Pâté chinois,903
Carnes,Tourtière,755
Mariscos,Boston Crab Meat,1103
Mariscos,Jack's New England Clam Chowder,981
Mariscos,Konbu,891


3)	Listar la cantidad de empleados por rango de antigüedad. Los rangos para considerar deben ser: 0 a 5 años, 5 a 10 años, 10 a 15 años y más de 15 años. Tener en cuenta que la lógica para determinar el rango de antigüedad debe escribirse una sola vez en la consulta. Se deberá ordenar por cantidad de mayor a menor.

In [6]:
WITH rango_antiguedad AS (
SELECT 
    CASE 
        WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 5 THEN '0-5 años'
        WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 10 THEN '5-10 años'
        WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 15 THEN '10-15 años'
        ELSE 'más de 15 años'
    END AS rango_antiguedad,
    IDEmpleado
    From Empleados
)
SELECT rango_antiguedad, COUNT(IDEmpleado) AS cantidad_empleados
FROM rango_antiguedad
GROUP BY rango_antiguedad
ORDER BY cantidad_empleados DESC;


(2 filas afectadas)

rango_antiguedad,cantidad_empleados
5-10 años,6
10-15 años,3


4)	Realizar la misma consulta con el agregado de que en el resultado deben aparecer todos los rangos.

In [7]:
WITH rangos AS (
    SELECT '0-5 años' AS rango_antiguedad
    UNION ALL
    SELECT '5-10 años'
    UNION ALL
    SELECT '10-15 años'
    UNION ALL
    SELECT 'más de 15 años'
), rango_antiguedad AS (
    SELECT 
        CASE 
            WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 5 THEN '0-5 años'
            WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 10 THEN '5-10 años'
            WHEN DATEDIFF(year, FechaAlta, GETDATE()) < 15 THEN '10-15 años'
            ELSE 'más de 15 años'
        END AS rango_antiguedad,
        IDEmpleado
    FROM Empleados
)
SELECT r.rango_antiguedad, COUNT(ra.IDEmpleado) AS cantidad_empleados
FROM rangos r
LEFT JOIN rango_antiguedad ra ON r.rango_antiguedad = ra.rango_antiguedad
GROUP BY r.rango_antiguedad
ORDER BY cantidad_empleados DESC;


(4 filas afectadas)

rango_antiguedad,cantidad_empleados
5-10 años,6
10-15 años,3
más de 15 años,0
0-5 años,0
